This notebook is written based on [this reference implementation](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/6%20-%20Transformers%20for%20Sentiment%20Analysis.ipynb).

Other refs for model:
* https://stackoverflow.com/questions/65205582/how-can-i-add-a-bi-lstm-layer-on-top-of-bert-model
* https://discuss.pytorch.org/t/how-to-connect-hook-two-or-even-more-models-together/21033
* https://pytorch.org/tutorials/beginner/transformer_tutorial.html
* https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

Other refs for torchtext:
* https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-i-5da6f1c89d84
* https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-ii-f146c8b9a496
* http://anie.me/On-Torchtext/

# Imports and setup

In [3]:
import pandas as pd
import numpy as np
import os
import random
# random.seed(1)
import re

# Data processing.
import constants # constants.py
import dataset # dataset.py
import torch

# Model.
import models # models.py
import torch.nn as nn

# Training.
import training # training.py
import utils # utils.py

# If you make a code change that doesn't get picked up by
# Jupyter notebook, try reloading like below:
# import imp
# imp.reload(training)

# Read the data
Skip this section if you've already ran the notebook once and have the csvs locally.

In [3]:
# data_df = dataset.read_multiple_datasets([1,2,3], 'Creativity_Combined', shuffle=True)

In [4]:
'''This cell is commented out because the csvs should already exist in the directory.
If you are running the notebook for the first time, run them to generate the csvs.'''
# split into train, test sets. (Train set will be further split into 
# train+validation sets, via k-fold CV.)
# train_df = data_df[:1000]
# test_df = data_df[1000:] # roughly 203 test examples set aside

# write them to CSV files
# train_df.to_csv('ktrain.csv', index=False, header=False)
# test_df.to_csv('ktest.csv', index=False, header=False)

## Preprocessing and transform into torchtext Dataset format.

From what I understand, some preprocessing is done when data.Field() is applied.

In [21]:
train_dataset, test_dataset = dataset.get_train_test_datasets()

In [22]:
# Transform train_dataset into an np array representation.
# This will be used for generating the K folds.
train_exs_arr = np.array(train_dataset.examples)

# Training pipeline begins here


In [32]:
import imp
imp.reload(training)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

params = {
    'dropout': 0.2,
    'batch_size': 8,
    'n_epochs': 3,
    'lr': 5e-05
}

# This experiment will not save any weights.
valid_corrs = training.launch_experiment(1, train_exs_arr, constants.N_SPLITS, params)
print('validation correlations: {}'.format(valid_corrs))

training on fold 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 00 | Epoch Time: 0m 49s
	 Train Loss: 8.622 | Train Corr: 0.22
	 Val. Loss: 4.616 |  Val. Corr: 0.48
Epoch: 01 | Epoch Time: 0m 46s
	 Train Loss: 2.875 | Train Corr: 0.62
	 Val. Loss: 3.015 |  Val. Corr: 0.62
Epoch: 02 | Epoch Time: 0m 45s
	 Train Loss: 2.238 | Train Corr: 0.72
	 Val. Loss: 3.333 |  Val. Corr: 0.62
training on fold 1


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 00 | Epoch Time: 0m 48s
	 Train Loss: 9.848 | Train Corr: 0.20
	 Val. Loss: 5.443 |  Val. Corr: 0.58
Epoch: 01 | Epoch Time: 0m 47s
	 Train Loss: 3.330 | Train Corr: 0.53
	 Val. Loss: 5.027 |  Val. Corr: 0.68
Epoch: 02 | Epoch Time: 0m 47s
	 Train Loss: 2.496 | Train Corr: 0.67
	 Val. Loss: 4.269 |  Val. Corr: 0.65
training on fold 2


...

Epoch: 00 | Epoch Time: 0m 48s
	 Train Loss: 8.632 | Train Corr: 0.22
	 Val. Loss: 5.713 |  Val. Corr: 0.50
Epoch: 01 | Epoch Time: 0m 46s
	 Train Loss: 3.621 | Train Corr: 0.51
	 Val. Loss: 5.273 |  Val. Corr: 0.65
Epoch: 02 | Epoch Time: 0m 46s
	 Train Loss: 2.775 | Train Corr: 0.65
	 Val. Loss: 4.530 |  Val. Corr: 0.78
training on fold 3


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 00 | Epoch Time: 0m 46s
	 Train Loss: 8.578 | Train Corr: 0.19
	 Val. Loss: 5.422 |  Val. Corr: 0.50
Epoch: 01 | Epoch Time: 0m 46s
	 Train Loss: 3.370 | Train Corr: 0.52
	 Val. Loss: 3.712 |  Val. Corr: 0.66
Epoch: 02 | Epoch Time: 0m 46s
	 Train Loss: 2.759 | Train Corr: 0.63
	 Val. Loss: 3.492 |  Val. Corr: 0.72
training on fold 4


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 00 | Epoch Time: 0m 44s
	 Train Loss: 10.511 | Train Corr: 0.13
	 Val. Loss: 5.077 |  Val. Corr: 0.52
Epoch: 01 | Epoch Time: 0m 43s
	 Train Loss: 3.350 | Train Corr: 0.53
	 Val. Loss: 4.717 |  Val. Corr: 0.57
Epoch: 02 | Epoch Time: 0m 44s
	 Train Loss: 2.428 | Train Corr: 0.69
	 Val. Loss: 5.262 |  Val. Corr: 0.53
validation correlations: [0.61981049 0.65365345 0.78285555 0.72438986 0.52795727]


# Test the trained model on held-out dataset.

In [11]:
# Get a test iterator
test_iterator = training.get_iterator(test_dataset, params['batch_size'], device)

In [12]:
# load the best model saved
bert = DistilBertModel.from_pretrained(constants.WEIGHTS_NAME)
model = models.BERTLinear(bert, constants.OUTPUT_DIM, params['dropout'])
model.load_state_dict(torch.load("best_valid_loss.pt"))
model.to(device)
model.eval()
# If you change the criterion, make sure it matches with the training criterion in training.py
criterion = nn.MSELoss(size_average=False)
criterion = criterion.to(device)
test_loss, test_corr = training.evaluate(model, test_iterator, criterion, debug=True)
print(test_loss)
print(test_corr)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


predictions: tensor([3.7420, 3.1371, 2.9669, 4.6738, 4.3069, 4.3898, 3.0036, 3.5888],
       device='cuda:0')
true labels: tensor([4.1750, 2.6750, 3.3500, 4.3250, 4.1750, 4.5750, 3.2500, 4.9500],
       device='cuda:0')
predictions: tensor([3.4298, 4.1198, 3.7915, 3.9154, 3.7454, 2.9854, 4.5348, 3.8341],
       device='cuda:0')
true labels: tensor([4.1250, 4.3500, 4.7000, 4.3000, 5.2750, 2.5500, 5.4250, 3.2750],
       device='cuda:0')
predictions: tensor([3.9121, 2.8993, 3.8251, 4.6524, 3.7348, 4.3551, 3.6334, 3.5260],
       device='cuda:0')
true labels: tensor([5.2750, 2.7750, 4.3750, 4.7750, 4.3750, 3.5500, 4.0500, 3.9250],
       device='cuda:0')
predictions: tensor([3.3758, 3.6365, 2.9481, 3.0048, 4.2327, 4.4699, 3.6686, 3.5043],
       device='cuda:0')
true labels: tensor([4.6000, 3.9250, 2.8500, 3.3000, 4.8000, 3.3250, 5.0500, 3.7250],
       device='cuda:0')
predictions: tensor([3.5478, 4.4148, 3.4240, 3.7057, 3.6455, 4.6826, 4.1336, 3.1818],
       device='cuda:0')
true label

# Misc other stuff

Link to the trainer class: https://huggingface.co/transformers/main_classes/trainer.html



Default training arguments: https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments

Batch size per device: 8

Epoch: 3



This should be the model I used to generate my initial results: https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
"DistilBert Model transformer with a sequence classification/regression head on top (a linear layer on top of the pooled output) e.g. for GLUE tasks."